# The largest prime (so far)

A new record for the largest prime has been found lately. Let explore this number and know more about it and how to deal with it using some Python, Numpy and finally using we will take a loog at Cython and GMP.

In [1]:
import numpy as np
import math
from datetime import datetime

%load_ext Cython

This number of was found in 2016 using the Great Internet Mersenne Prime Search (GIMPS). This prime is one of the Mersenne Prime whice are of the defined by the following formula:

$$2^p-1$$

Where $p$ is a prime number. This is called a psudo-prime meaning not all Mersenne Prime are primes.

$2^2-1=3$

$2^3-1=7$

$2^5-1=31$

$2^7-1=127$   ...


In [2]:
2**2-1

3

In [3]:
2**3-1

7

In [4]:
2**5-1

31

In [5]:
2**7-1

127

The smallest none-prime (composite) number is $2^{11}-1=2047$ which is a composite number.

In [7]:
2**11-1

2047

The largest prime number is that was found lately is:
$$2^{74,207,281}-1$$

The reason why I didn't put the actual value is that the number if very large (over 22 million digits).

In [9]:
p = 74207281
the_number = (2 ** p) - 1

# Lucasâ€“Lehmer test

A very easy way to test a Mersenne Prime is the Lucas-Lehmer test which uses the Lucas series.

$4, 14, 37634 ... $

The series starts with 4 then uses the last value in this formula:

$S_i=S_{i-1}^2-2$

In [10]:
S = 4
print(S)
print(len(str(S)))

4
1


In [11]:
S = S ** 2 - 2
print(S)
print(len(str(S)), "digits")

14
2 digits


In [12]:
S = S ** 2 - 2
print(S)
print(len(str(S)), "digits")

194
3 digits


In [13]:
S = S ** 2 - 2
print(S)
print(len(str(S)), "digits")

37634
5 digits


In [14]:
S = S ** 2 - 2
print(S)
print(len(str(S)), "digits")

1416317954
10 digits


In [15]:
S = S ** 2 - 2
print(S)
print(len(str(S)), "digits")

2005956546822746114
19 digits


In [16]:
# The largest prime (so far)
the_number = 2 ** 74207281 - 1
print(int(math.log10(the_number))+1, "digits")

22338618 digits


The test is if the Mersenne Prime number of $n^2-1$ is a divided by $S_{n-2}$ you should get a remaining of 0. The problem is this series goes very large very fast. So there is another way to do this test.

In [20]:
p = 74207281
the_number = (2 ** p) - 1

S = 4
time_stamp = datetime.now()
for i in range(p-2):
    S = (S ** 2 - 2) % the_number
    if i % 1 == 0:
        print(i, datetime.now() - time_stamp,"")
        time_stamp = datetime.now()
if S == 0:
    print("PRIME")

0 0:00:00.000423 
1 0:00:00.000017 
2 0:00:00.000012 
3 0:00:00.000014 
4 0:00:00.000014 
5 0:00:00.000013 
6 0:00:00.000014 
7 0:00:00.000014 
8 0:00:00.000015 
9 0:00:00.000014 
10 0:00:00.000036 
11 0:00:00.000051 
12 0:00:00.000127 
13 0:00:00.000358 
14 0:00:00.001040 
15 0:00:00.002735 
16 0:00:00.004968 
17 0:00:00.014782 
18 0:00:00.044420 
19 0:00:00.132890 
20 0:00:00.398497 
21 0:00:01.198042 
22 0:00:03.592832 
23 0:00:10.796848 


KeyboardInterrupt: 

In [21]:
%%cython
cdef unsigned long p = 61
cdef unsigned long P = (2 ** p) - 1

S = 4
for i in range(p-2):
    S = S ** 2 - 2
    S = S % P
    if i % 10 == 0:
        print(i)
if S == 0:
    print("PRIME")

0
10
20
30
40
50
PRIME


/usr/local/lib/python3.4/dist-packages/IPython/utils/path.py:264: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module
  warn("get_ipython_cache_dir has moved to the IPython.paths module")


In [ ]:
%%cython --link-args=-lgmp

cdef extern from "gmp.h":
    ctypedef struct mpz_t:
        pass
    
    ctypedef unsigned long mp_bitcnt_t
    
    cdef void mpz_init(mpz_t)
    cdef void mpz_init_set_ui(mpz_t, unsigned int)
    
    cdef void mpz_add(mpz_t, mpz_t, mpz_t)
    cdef void mpz_add_ui(mpz_t, const mpz_t, unsigned long int)
    
    cdef void mpz_sub (mpz_t, const mpz_t, const mpz_t)
    cdef void mpz_sub_ui (mpz_t, const mpz_t, unsigned long int)
    cdef void mpz_ui_sub (mpz_t, unsigned long int, const mpz_t)
    
    cdef void mpz_mul (mpz_t, const mpz_t, const mpz_t)
    cdef void mpz_mul_si (mpz_t, const mpz_t, long int)
    cdef void mpz_mul_ui (mpz_t, const mpz_t, unsigned long int)
    
    cdef void mpz_mul_2exp (mpz_t, const mpz_t, mp_bitcnt_t)
    
    cdef void mpz_mod (mpz_t, const mpz_t, const mpz_t)
    
    cdef unsigned long int mpz_get_ui(const mpz_t)

#cdef unsigned long p = 61
cdef mp_bitcnt_t p = 74207281
cdef mpz_t t # = 1
cdef mpz_t a # = 1
cdef mpz_t P # = (2 ** p) - 1
cdef mpz_t S # = 4

mpz_init(t)
mpz_init_set_ui(t, 1)

mpz_init(a)
mpz_init_set_ui(a, 2)

mpz_init(P)
mpz_mul_2exp(P,t,p)
mpz_sub_ui(P,P,1)

mpz_init(S)
mpz_init_set_ui(S, 4)

for i in range(p-2):
    #S = S ** 2 - 2
    mpz_mul(S,S,S)
    mpz_sub_ui(S,S,2)
    
    #S = S % P
    mpz_mod(S,S,P)
    
    if i % 1000 == 0:
        print(i)
if mpz_get_ui(S) == 0:
    print("PRIME")
else:
    print("COMPOSITE")

#print(mpz_get_ui(P))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000